# 03 — Baselines & Heuristics

**Objectif :** implémenter des méthodes de référence (baselines) pour l’ordonnancement,
afin de comparer ensuite les méthodes Monte Carlo.

Baselines implémentées :
- Random (planning aléatoire)
- Greedy list scheduling (assignation sur la machine la plus tôt disponible)
- Variantes avec règles de tri : SPT (durées courtes d’abord), EDD (deadline la plus proche)


In [1]:
import json
import numpy as np
import random
import os
import sys, pathlib

PROJECT_ROOT = pathlib.Path.cwd().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

INSTANCES_DIR = PROJECT_ROOT / "data" / "instances"
print("INSTANCES_DIR:", INSTANCES_DIR)


INSTANCES_DIR: /data/instances


## 1) Chargement d’une instance et fonctions utilitaires

On réutilise :
- validation de solution
- simulation
- calcul de métriques / score


In [2]:
def load_instance(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def is_valid_solution(solution, n_jobs):
    flat = [job for seq in solution for job in seq]
    return (len(flat) == n_jobs) and (sorted(flat) == list(range(n_jobs)))

def simulate(instance, solution):
    n_jobs = instance["n_jobs"]
    n_machines = instance["n_machines"]
    p = instance["processing_times"]
    releases = instance.get("releases", None)
    if releases is None:
        releases = [0.0] * n_jobs

    start = [0.0] * n_jobs
    end = [0.0] * n_jobs
    machine_time = [0.0] * n_machines

    for k in range(n_machines):
        t = 0.0
        for j in solution[k]:
            t = max(t, float(releases[j]))
            start[j] = t
            t = t + float(p[j][k])
            end[j] = t
        machine_time[k] = t

    Cmax = max(end) if end else 0.0
    return {"start": start, "end": end, "machine_time": machine_time, "Cmax": Cmax}

def compute_metrics(instance, sim, alpha=1.0, beta=0.2, gamma=0.0):
    end = sim["end"]
    Cmax = float(sim["Cmax"])

    deadlines = instance.get("deadlines", None)
    weights = instance.get("weights", None)

    tardiness = [0.0] * instance["n_jobs"]
    weighted_tardiness = [0.0] * instance["n_jobs"]

    sumT = 0.0
    sumWT = 0.0

    if deadlines is not None:
        for j in range(instance["n_jobs"]):
            tj = max(0.0, float(end[j]) - float(deadlines[j]))
            tardiness[j] = tj
            sumT += tj

            if weights is not None:
                wtj = float(weights[j]) * tj
                weighted_tardiness[j] = wtj
                sumWT += wtj

    score = alpha * Cmax + beta * sumT + gamma * sumWT

    return {
        "Cmax": Cmax,
        "sumT": sumT,
        "sumWT": sumWT,
        "score": score
    }


In [4]:
instance_path = "/content/data/instances/instance_50jobs_5machines.json"
instance = load_instance(instance_path)

n_jobs = instance["n_jobs"]
n_machines = instance["n_machines"]
print("Loaded:", instance["name"], "| n_jobs:", n_jobs, "| n_machines:", n_machines)
print("deadlines:", instance.get("deadlines") is not None)

Loaded: instance_50jobs_5machines | n_jobs: 50 | n_machines: 5
deadlines: True


## 2) Baseline 1 — Random scheduling

On génère un planning aléatoire :
- assignation machine aléatoire pour chaque tâche
- ordre aléatoire sur chaque machine

Cette baseline donne une référence faible mais utile.


In [5]:
def random_solution(instance, seed=0):
    rng = random.Random(seed)
    n_jobs = instance["n_jobs"]
    n_machines = instance["n_machines"]

    jobs = list(range(n_jobs))
    rng.shuffle(jobs)

    sol = [[] for _ in range(n_machines)]
    for j in jobs:
        k = rng.randrange(n_machines)
        sol[k].append(j)

    for k in range(n_machines):
        rng.shuffle(sol[k])

    return sol


## 3) Baseline 2 — Greedy list scheduling

Principe :
- on choisit un ordre global des tâches (list) via une règle
- on place chaque tâche sur la machine qui **termine le plus tôt**

Règles de tri :
- **SPT** (Shortest Processing Time) : durées courtes d’abord
- **EDD** (Earliest Due Date) : deadlines les plus proches d’abord

Pour SPT, on utilise une durée représentative par tâche :
\[
\bar{p}_j = \frac{1}{m}\sum_{k} p_{j,k}
\]


In [6]:
def job_order(instance, rule="SPT"):
    n_jobs = instance["n_jobs"]
    p = np.array(instance["processing_times"], dtype=float)
    deadlines = instance.get("deadlines", None)

    if rule.upper() == "SPT":
        avg_p = p.mean(axis=1)  # durée moyenne par job
        order = list(np.argsort(avg_p))
        return order

    if rule.upper() == "EDD":
        if deadlines is None:
            raise ValueError("EDD nécessite des deadlines dans l'instance.")
        order = list(np.argsort(np.array(deadlines, dtype=float)))
        return order

    raise ValueError("rule doit être 'SPT' ou 'EDD'.")


In [7]:
def greedy_list_scheduling(instance, rule="SPT"):
    n_jobs = instance["n_jobs"]
    n_machines = instance["n_machines"]
    p = instance["processing_times"]
    releases = instance.get("releases", None)
    if releases is None:
        releases = [0.0] * n_jobs

    order = job_order(instance, rule=rule)
    sol = [[] for _ in range(n_machines)]
    machine_time = [0.0] * n_machines  # fin actuelle de chaque machine

    for j in order:
        # choisir machine qui finit le plus tôt si on place j ensuite
        best_k = None
        best_finish = float("inf")
        for k in range(n_machines):
            start_j = max(machine_time[k], float(releases[j]))
            finish_j = start_j + float(p[j][k])
            if finish_j < best_finish:
                best_finish = finish_j
                best_k = k

        sol[best_k].append(j)
        machine_time[best_k] = best_finish

    return sol


## 4) Évaluation d’une méthode

On définit une fonction qui :
- construit une solution
- simule
- calcule les métriques (Cmax, sumT, score)

On fixera les poids du score : $\alpha=1$, $\beta=0.2$, $\gamma=0$ par défaut.


In [8]:
def evaluate_solution(instance, sol, alpha=1.0, beta=0.2, gamma=0.0):
    assert is_valid_solution(sol, instance["n_jobs"]), "Solution invalide"
    sim = simulate(instance, sol)
    met = compute_metrics(instance, sim, alpha=alpha, beta=beta, gamma=gamma)
    return met


In [9]:
alpha, beta, gamma = 1.0, 0.2, 0.0

sol_rand = random_solution(instance, seed=0)
met_rand = evaluate_solution(instance, sol_rand, alpha, beta, gamma)

sol_spt = greedy_list_scheduling(instance, rule="SPT")
met_spt = evaluate_solution(instance, sol_spt, alpha, beta, gamma)

sol_edd = greedy_list_scheduling(instance, rule="EDD") if instance.get("deadlines") is not None else None
met_edd = evaluate_solution(instance, sol_edd, alpha, beta, gamma) if sol_edd is not None else None

print("Random:", met_rand)
print("Greedy SPT:", met_spt)
if met_edd is not None:
    print("Greedy EDD:", met_edd)


Random: {'Cmax': 157.0, 'sumT': 372.46835587990915, 'sumWT': 0.0, 'score': 231.49367117598183}
Greedy SPT: {'Cmax': 59.0, 'sumT': 0.0, 'sumWT': 0.0, 'score': 59.0}
Greedy EDD: {'Cmax': 51.0, 'sumT': 0.0, 'sumWT': 0.0, 'score': 51.0}


## 5) Évaluation multi-seeds (robuste)

Pour comparer les baselines de manière fiable, on répète l’évaluation sur plusieurs seeds
(pour la méthode random) et on observe la moyenne et l’écart-type.

Les heuristiques déterministes (SPT, EDD) donnent le même résultat à chaque exécution.


In [10]:
def stats(values):
    values = np.array(values, dtype=float)
    return {
        "mean": float(values.mean()),
        "std": float(values.std(ddof=1)) if len(values) > 1 else 0.0,
        "min": float(values.min()),
        "max": float(values.max()),
    }

seeds = list(range(30))

scores_rand = []
cmax_rand = []
sumT_rand = []

for s in seeds:
    sol = random_solution(instance, seed=s)
    met = evaluate_solution(instance, sol, alpha, beta, gamma)
    scores_rand.append(met["score"])
    cmax_rand.append(met["Cmax"])
    sumT_rand.append(met["sumT"])

rand_stats = {
    "score": stats(scores_rand),
    "Cmax": stats(cmax_rand),
    "sumT": stats(sumT_rand),
}

spt_stats = evaluate_solution(instance, greedy_list_scheduling(instance, "SPT"), alpha, beta, gamma)
edd_stats = evaluate_solution(instance, greedy_list_scheduling(instance, "EDD"), alpha, beta, gamma) if instance.get("deadlines") is not None else None

rand_stats, spt_stats, edd_stats


({'score': {'mean': 253.76838712306395,
   'std': 58.665782074659525,
   'min': 162.2179529055988,
   'max': 397.9232563919876},
  'Cmax': {'mean': 158.06666666666666,
   'std': 23.96395761070813,
   'min': 123.0,
   'max': 209.0},
  'sumT': {'mean': 478.5086022819864,
   'std': 183.22905350144657,
   'min': 196.08976452799388,
   'max': 944.6162819599377}},
 {'Cmax': 59.0, 'sumT': 0.0, 'sumWT': 0.0, 'score': 59.0},
 {'Cmax': 51.0, 'sumT': 0.0, 'sumWT': 0.0, 'score': 51.0})

## 6) Résumé comparatif (baseline vs heuristiques)

On résume les performances dans un tableau :
- Random : moyenne ± écart-type (sur plusieurs seeds)
- SPT / EDD : valeur unique (déterministes)


In [11]:
import pandas as pd

rows = []

rows.append({
    "Method": "Random (mean)",
    "Score": rand_stats["score"]["mean"],
    "Cmax": rand_stats["Cmax"]["mean"],
    "sumT": rand_stats["sumT"]["mean"],
})

rows.append({
    "Method": "Random (std)",
    "Score": rand_stats["score"]["std"],
    "Cmax": rand_stats["Cmax"]["std"],
    "sumT": rand_stats["sumT"]["std"],
})

rows.append({
    "Method": "Greedy SPT",
    "Score": spt_stats["score"],
    "Cmax": spt_stats["Cmax"],
    "sumT": spt_stats["sumT"],
})

if edd_stats is not None:
    rows.append({
        "Method": "Greedy EDD",
        "Score": edd_stats["score"],
        "Cmax": edd_stats["Cmax"],
        "sumT": edd_stats["sumT"],
    })

df = pd.DataFrame(rows)
df


,Method,Score,Cmax,sumT
0,Random (mean),253.768387,158.066667,478.508602
1,Random (std),58.665782,23.963958,183.229054
2,Greedy SPT,59.000000,59.000000,0.000000
3,Greedy EDD,51.000000,51.000000,0.000000


## 7) Conclusion

Dans ce notebook, nous avons implémenté et évalué plusieurs méthodes de référence
pour le problème d’ordonnancement étudié.

La méthode **Random** fournit une baseline volontairement faible, caractérisée par
un makespan élevé et une forte variance, ce qui met en évidence la difficulté du
problème et la nécessité de méthodes plus structurées.

Les heuristiques gloutonnes **SPT** (Shortest Processing Time) et **EDD**
(Earliest Due Date) obtiennent des performances nettement supérieures.
En particulier, la règle **EDD** exploite efficacement l’information sur les deadlines,
ce qui conduit à des plannings sans retard et à un makespan réduit sur l’instance testée.

Ces résultats montrent que les heuristiques classiques peuvent être très efficaces
sur des instances simples ou bien structurées. Toutefois, leur performance dépend
fortement des caractéristiques de l’instance et elles peuvent devenir sous-optimales
lorsque le problème se complexifie.

Dans la suite du projet, nous utiliserons ces méthodes comme **références**
afin d’évaluer la capacité des approches **Monte Carlo** à explorer plus globalement
l’espace des solutions et à produire des plannings de meilleure qualité ou plus robustes
sur des instances plus difficiles.
